In [1]:
import os
import heartpy as hp
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import traceback
from joblib import Parallel, delayed

os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from src.parsing.MESAParser import MESAParser

#run the analysis
dl = MESAParser()
database_all_patient_IDs = dl.database_all_patient_IDs

def process(patientid):
    try: 
        path = dl.path_ppg_annotation(patientid, 'heartpy', 'Peaks')
        if os.path.isfile(path): 
            return
       
        signal = dl.load_signal(patientid, 'Pleth')
        signal_ = hp.filter_signal(signal, cutoff = 8, sample_rate = 256.0, order = 3, filtertype='lowpass')

        s = 100*256
        h = 90*256

        all_peaks = np.array([])
        for i in (range(int(len(signal)/h) -1)):
            try:
                wd, m = hp.process(signal_[i*h:i*h+s], sample_rate=256)
                peaks = np.array([w for w in wd['peaklist'] if w not in wd['removed_beats']])
                all_peaks = np.append(all_peaks, peaks+int(i*h))
            except:
                pass
        all_peaks_ = remove_duplicates(all_peaks)
        np.save(path, all_peaks_)
    except:
        print(f"Failed{patientid}")
    
def remove_duplicates(peaks):
    peaks = np.sort(peaks)
    dpeaks = np.diff(peaks)
    peaks = peaks[1:][dpeaks > 10]
    return peaks


# Parallel(n_jobs=32, prefer="threads")(delayed(process)(patient) for patient in tqdm.tqdm(database_all_patient_IDs, desc='Heartpy'))

for patient in tqdm.tqdm(database_all_patient_IDs, desc='Heartpy'): 
    process(patient)

Heartpy: 100%|██████████| 2052/2052 [14:02:16<00:00, 24.63s/it]  
